## MDPs in pymdptoolbox - praticals (github@pucrs-automated-planning/mdp)

In this practical exercise, we will look at how MDP planning is implemented in a mathematical toolkit, and track the calculation of the rewards for each state via Value Iteration. The following code sets up an MDP environment (the basic case shown in class, shown in the Figure below) and computes the policy for the given MDP using the Value Iteration algorithm.

<img align="center" src="./figures/maze-example.png" width=300/>
Then we provide a set of questions for you to implement and answer. 
This assignment is not graded.



In [4]:
# The line below is to be used if you have pymdptoolbox installed with setuptools
# import mdptoolbox.example
# Whereas the line below obviate the need to install that
import sys
sys.path.insert(0, 'pymdptoolbox/src')
import mdptoolbox.example

import numpy as _np
from gen_scenario import *

"""
(Y,X)
| 00 01 02 ... 0X-1       'N' = North
| 10  .         .         'S' = South
| 20    .       .         'W' = West
| .       .     .         'E' = East
| .         .   .         'T' = Terminal
| .           . .         'O' = Obstacle
| Y-1,0 . . .   Y-1X-1
""" 

shape = [3,4]
rewards = [[0,3,1],[1,3,-1]]
obstacles = [[1,1]]
terminals = [[0,3],[1,3]] # note this states are considered as terminal states
P, R = mdp_grid(shape=shape, terminals=terminals, r=-0.1, rewards=rewards, obstacles=obstacles)
vi = mdptoolbox.mdp.ValueIteration(P, R, discount=0.99, epsilon=0.001, max_iter=1000, skip_check=True)
vi.run()
#You can check the quadrant values using print vi.V
print_policy(vi.policy, shape, obstacles=obstacles, terminals=terminals)


[['E' 'E' 'E' 'T']
 ['N' 'O' 'N' 'T']
 ['N' 'W' 'N' 'W']]


Before you go for the questionnaire, take your time to open the sourcecode of the MDP toolkit we use, specifically, look into these files:
1. [gen_scenario.py](gen_scenario.py) - contains the conversion code to make the simple coordinate commands above (e.g. ```shape = [3,4]```) into the matrices actually used by the MDP solver
2. [mdp.py](pymdptoolbox/src/mdptoolbox/mdp.py) - contains most of the logic for an MDP, including the *Bellman Equation* as follows:

$$V(s) = \max_{a} \left[ R(s,a)+ \gamma \sum_{s'}P(s'|s,a)*V(s') \right]$$

See if you can identify where how this equation is implemented in the ```MDP._bellmanOperator``` with the [mdp.py](pymdptoolbox/src/mdptoolbox/mdp.py) file. Note how this implementation uses matrix multiplication to achieve the summation step described in the equation. Once you believe you understand that, go ahead and respond the questionnaire. 

### Questionnaire
1. Study the code of the cell above and answer the following questions.
	1. What is the policy generated if we change the discount factor of the grid domain to 0.1?
	2. Use the following line ```vi.verbose = True	``` before  ```vi.run()```:   
	What is the variation for each of the first three iterations with the discount factor of 0.9 and how many iterations does the algorithm take to converge?
	3. How does changes to the discount factor affect the variation of the state values over time?





In [8]:
#1.A
vi = mdptoolbox.mdp.ValueIteration(P, R, discount=0.1, epsilon=0.001, max_iter=1000, skip_check=True)
vi.verbose = True
vi.run()
#You can check the quadrant values using print vi.V
print_policy(vi.policy, shape, obstacles=obstacles, terminals=terminals)

 Iteration   Variation
         1    2.000000
         2    0.096800
         3    0.009152
Iterating stopped due to maximum number of iterations condition.
[['E' 'E' 'E' 'T']
 ['N' 'O' 'W' 'T']
 ['N' 'N' 'N' 'S']]


Converge fast but is dumb

In [9]:
#1.B
vi = mdptoolbox.mdp.ValueIteration(P, R, discount=0.9, epsilon=0.001, max_iter=1000, skip_check=True)
vi.verbose = True
vi.run()
#You can check the quadrant values using print vi.V
print_policy(vi.policy, shape, obstacles=obstacles, terminals=terminals)

 Iteration   Variation
         1    2.000000
         2    0.871200
         3    0.741312
         4    0.574802
         5    0.407370
         6    0.340194
         7    0.203374
         8    0.114344
         9    0.055497
        10    0.026025
        11    0.011618
        12    0.005090
        13    0.002180
        14    0.000924
        15    0.000387
        16    0.000161
        17    0.000066
Iterating stopped, epsilon-optimal policy found.
[['E' 'E' 'E' 'T']
 ['N' 'O' 'N' 'T']
 ['N' 'E' 'N' 'W']]


1    2.000000
2    0.871200
3    0.741312

Need 17 iterations

#1.C 

More the dicount is high more the variation is low.

# 2. 
The scenario below has an interesting structure whereby the positive rewarding terminal state is partially surrounded by negatively-rewarding states. Program this scenario in pymdptoolbox and compute the optimal policy with a discount factor of 0.99.

<img align="center" src="./figures/maze-example-2.png" width=300/>



In [12]:
#2
shape = [5,5]
rewards = [[1,2,-1],[2,1,-1],[2,2,1],[3,2,-1]]
obstacles = [[1,1],[1,3],[3,1],[3,3]]
terminals = [[1,2],[2,1],[2,2],[3,2]] # note this states are considered as terminal states
P, R = mdp_grid(shape=shape, terminals=terminals, r=-0.1, rewards=rewards, obstacles=obstacles)
vi = mdptoolbox.mdp.ValueIteration(P, R, discount=0.99, epsilon=0.001, max_iter=1000, skip_check=True)
vi.run()
#You can check the quadrant values using print vi.V
print_policy(vi.policy, shape, obstacles=obstacles, terminals=terminals)


[['E' 'E' 'E' 'E' 'S']
 ['N' 'O' 'T' 'O' 'S']
 ['N' 'T' 'T' 'W' 'W']
 ['S' 'O' 'T' 'O' 'N']
 ['E' 'E' 'E' 'E' 'N']]


3. Define two new 5 by 5 scenarios with multiple obstacles and an interesting geometry following the guidelines below. Calculate the policy with discount factor 0.99, and then try to explain intuitively the reason for the resulting policies, given the initial parameters. These two scenarios must have the following characteristics:
	1. A scenario with one (or more) terminal states with positive rewards and at least one other state with the same amount of, but negative reward and no terminal states with negative rewards.
	2. A scenario with one terminal state with a negative reward and at least one non-terminal state with a positive reward.

In [13]:
#3.A
shape = [5,5]
rewards = [[1,2,-1],[2,1,-1],[2,2,1],[3,2,-1]]
obstacles = [[1,1],[1,3],[3,1],[3,3]]
terminals = [[2,2]] # note this states are considered as terminal states
P, R = mdp_grid(shape=shape, terminals=terminals, r=-0.1, rewards=rewards, obstacles=obstacles)
vi = mdptoolbox.mdp.ValueIteration(P, R, discount=0.99, epsilon=0.001, max_iter=1000, skip_check=True)
vi.run()
#You can check the quadrant values using print vi.V
print_policy(vi.policy, shape, obstacles=obstacles, terminals=terminals)

[['E' 'E' 'E' 'E' 'S']
 ['N' 'O' 'S' 'O' 'S']
 ['E' 'E' 'T' 'W' 'W']
 ['S' 'O' 'N' 'O' 'N']
 ['E' 'E' 'E' 'E' 'N']]


In [15]:
#3.B
shape = [5,5]
rewards = [[1,2,1],[2,1,1],[2,2,1],[3,2,-1]]
obstacles = [[1,1],[1,3],[3,1],[3,3]]
terminals = [[3,2]] # note this states are considered as terminal states
P, R = mdp_grid(shape=shape, terminals=terminals, r=-0.1, rewards=rewards, obstacles=obstacles)
vi = mdptoolbox.mdp.ValueIteration(P, R, discount=0.99, epsilon=0.001, max_iter=1000, skip_check=True)
vi.run()
#You can check the quadrant values using print vi.V
print_policy(vi.policy, shape, obstacles=obstacles, terminals=terminals)

[['S' 'E' 'S' 'W' 'W']
 ['S' 'O' 'S' 'O' 'S']
 ['E' 'E' 'N' 'W' 'W']
 ['N' 'O' 'T' 'O' 'N']
 ['N' 'W' 'S' 'E' 'N']]


4. Chose one of previous scenarios, and apply mdptoolbox.mdp.PolicyIteration.

    A . How many iterations are needed to reach the optimal policy ?

In [ ]:
#4.A